## Introduction

In the previous lab, we learned that PCA performs dimensionality reduction by finding linear combinations of original features. While effective for linear relationships, PCA cannot capture complex non-linear patterns in data.

**Kernel PCA** extends PCA to handle non-linear relationships by applying kernel methods. However, it has two main limitations:
- Computationally expensive for large datasets
- Requires careful selection of kernel type and parameters

In this lab, we explore two powerful alternatives for visualizing high-dimensional data:

- **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: Excels at preserving local structure and revealing clusters, making it ideal for visualization
- **UMAP (Uniform Manifold Approximation and Projection)**: Balances both local and global structure preservation while being computationally efficient

We'll apply these methods to the MNIST handwritten digit dataset and compare their performance against PCA and Kernel PCA. Through this comparison, you'll understand the strengths, computational trade-offs, and appropriate use cases for each dimensionality reduction technique.

## Learning Goals

By the end of this lab, you will be able to:

* Compare and contrast different dimensionality reduction approaches (PCA, Kernel PCA, t-SNE, UMAP) using the MNIST dataset
* Optimize t-SNE visualizations by tuning key hyperparameters: perplexity, initialization methods, and early exaggeration
* Optimize UMAP visualizations by adjusting n_neighbors, min_dist, and distance metrics
* Understand the strengths, limitations, and appropriate use cases for each dimensionality reduction method
* Make informed decisions about which technique to use based on dataset characteristics and computational constraints

## MNIST Dataset

The MNIST dataset is a collection of 70,000 handwritten digit images, each represented as a 28x28 pixel grayscale image (784 dimensions). This high-dimensional dataset serves as an excellent benchmark for comparing dimensionality reduction techniques, as the underlying structure corresponds to the ten digit classes (0-9). We will visualize how different methods capture and preserve this structure in lower-dimensional spaces.

In [ ]:
# import commonly used libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# import the data
import sklearn.datasets

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import squareform, pdist
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#read mnist dataset from sklearn
mnist = sklearn.datasets.fetch_openml('mnist_784', version=1)

# show the shape of the data
print(mnist.data.shape)

# Full dataset references (use full MNIST everywhere except Kernel PCA sample)
X_full = mnist.data
y_full = mnist.target.astype(int)

In [ ]:
# below are drawings of some of the digits in the dataset
mnist_names = [i for i in range(10)]

plt.figure(figsize=(14,10))
for i in range(40):
    plt.subplot(5, 8, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.array(mnist.data.iloc[i]).reshape((28, 28)), cmap=plt.cm.binary)
    plt.xlabel(mnist_names[int(mnist.target[i])])
plt.show()

## Principal Component Analysis (PCA)

PCA is a linear dimensionality reduction technique that identifies orthogonal axes of maximum variance. While computationally efficient, PCA may not capture complex non-linear relationships in the data. We will reduce the MNIST dataset to 2 components and examine the resulting visualization.

In [ ]:
%%time
# Fit PCA to 2 components and transform the data
pca = PCA(n_components=2, whiten=True, random_state=42)
mnist_pca = pca.fit_transform(mnist.data)

# show the culmulative explained variance using 2 components
print('Explained variance: ', np.sum(pca.explained_variance_ratio_))

# plot the data
plt.figure(figsize=(10, 7))
plt.scatter(mnist_pca[:, 0], mnist_pca[:, 1], c=mnist.target.astype(int), cmap='tab10', s=8)
plt.colorbar()
plt.xlabel('PCA First Component')
plt.ylabel('PCA Second Component')
plt.title('PCA')
plt.show()

## Kernel PCA

Kernel PCA extends traditional PCA by applying the kernel trick to capture non-linear patterns in data. By using an RBF (Radial Basis Function) kernel, we can identify complex curved structures that linear PCA would miss.

**Computational Considerations**: Kernel PCA has O(n²) to O(n³) complexity, making it expensive for large datasets. For this demonstration, we use a random sample of the MNIST data rather than all 70,000 points to maintain reasonable runtime while still illustrating the method's capabilities.

In [ ]:
%%time
# Kernel PCA: use a smaller random sample due to computational cost
from sklearn.decomposition import KernelPCA
sample_size = 35000  # adjust for performance vs quality
rng = np.random.RandomState(42)
sample_idx = rng.choice(len(X_full), size=sample_size, replace=False)
X_kpca = X_full.iloc[sample_idx]
y_kpca = y_full[sample_idx]
kpca = KernelPCA(n_components=2, kernel='rbf', gamma=0.03)
mnist_kpca = kpca.fit_transform(X_kpca)

plt.figure(figsize=(10, 7))
plt.scatter(mnist_kpca[:, 0], mnist_kpca[:, 1], c=y_kpca, cmap='tab10', s=8)
plt.colorbar()
plt.xlabel('Kernel PCA First Component')
plt.ylabel('Kernel PCA Second Component')
plt.title(f'Kernel PCA (sample of {sample_size} points)')
plt.show()

## t-Distributed Stochastic Neighbor Embedding (t-SNE)

t-SNE is a non-linear dimensionality reduction technique specifically designed for visualization. It excels at preserving local structure and revealing cluster patterns in high-dimensional data.

**How t-SNE Works**:
1. Converts high-dimensional Euclidean distances into conditional probabilities representing pairwise similarities
2. Creates a similar probability distribution in the low-dimensional space
3. Minimizes the divergence between these two distributions using gradient descent

**Key Characteristics**:
- **Strengths**: Excellent at revealing clusters and local neighborhoods; produces highly interpretable visualizations
- **Limitations**: Can be computationally intensive on large datasets; primarily designed for visualization rather than general dimensionality reduction
- **Performance Note**: Running t-SNE on the full MNIST dataset (70,000 points) takes considerable time with sklearn's implementation. For faster experimentation, consider using a random sample or exploring optimized implementations like openTSNE or MulticoreTSNE.

In [ ]:
%%time

# using t-SNE from sklearn on the entire dataset, it will take some time, feel free to take a random sample subset for your experiment
tsne = TSNE(n_components=2, random_state=42)
mnist_tsne = tsne.fit_transform(X_full)

plt.figure(figsize=(10, 7))
plt.scatter(mnist_tsne[:, 0], mnist_tsne[:, 1], c=y_full, cmap='tab10', s=8)
plt.colorbar()
plt.xlabel('t-SNE First Component')
plt.ylabel('t-SNE Second Component')
plt.title('t-SNE (Full MNIST)')
plt.show()

### Hyperparameter Tuning for t-SNE

The quality of t-SNE visualizations depends heavily on hyperparameter selection. We will explore three key parameters:

1. **Perplexity**: Controls the balance between local and global structure. Lower values (5-10) emphasize local patterns, while higher values (50-100) consider broader neighborhoods. Typical range: 5-50 for most datasets.

2. **Initialization**: Starting point for optimization. PCA initialization typically provides more stable and interpretable results, while random initialization may reveal alternative structure.

3. **Early Exaggeration**: Amplifies distances in early optimization to help form distinct clusters. Higher values create tighter, more separated clusters but may overemphasize divisions.

In [ ]:
# Perplexity sweep on full dataset (caution: very slow overall)
perplexities = [5, 20, 50, 100]

plt.figure(figsize=(20, 10))
for i, perplexity in enumerate(perplexities):
    plt.subplot(2, 2, i+1)
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    mnist_tsne = tsne.fit_transform(X_full)
    plt.scatter(mnist_tsne[:, 0], mnist_tsne[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('t-SNE First Component')
    plt.ylabel('t-SNE Second Component')
    plt.title(f't-SNE perplexity {perplexity}')
plt.show()

In [ ]:
# Initialization comparison (PCA vs random) on full dataset
plt.figure(figsize=(20, 10))
for i, init in enumerate(['pca', 'random']):
    plt.subplot(1, 2, i+1)
    tsne = TSNE(n_components=2, random_state=42, perplexity=50, init=init)
    mnist_tsne = tsne.fit_transform(X_full)
    plt.scatter(mnist_tsne[:, 0], mnist_tsne[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('t-SNE First Component')
    plt.ylabel('t-SNE Second Component')
    plt.title(f't-SNE init={init}')
plt.show()

In [ ]:
# Early exaggeration sweep on full dataset
early_exaggerations = [2, 5, 10, 50]

plt.figure(figsize=(20, 10))
for i, early_exaggeration in enumerate(early_exaggerations):
    plt.subplot(2, 2, i+1)
    tsne = TSNE(n_components=2, random_state=42, early_exaggeration=early_exaggeration)
    mnist_tsne = tsne.fit_transform(X_full)
    plt.scatter(mnist_tsne[:, 0], mnist_tsne[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('t-SNE First Component')
    plt.ylabel('t-SNE Second Component')
    plt.title(f't-SNE early_exaggeration={early_exaggeration}')
plt.show()

## Uniform Manifold Approximation and Projection (UMAP)

UMAP is a modern dimensionality reduction technique that balances preserving both local and global structure. It constructs a high-dimensional graph representation of the data, then optimizes a low-dimensional layout. UMAP typically runs faster than t-SNE and often produces more meaningful global structure while maintaining cluster separation.

In [ ]:
%%time
# UMAP on full dataset
import umap
umap1 = umap.UMAP(n_components=2, random_state=42)
mnist_umap = umap1.fit_transform(X_full)

plt.figure(figsize=(10, 7))
plt.scatter(mnist_umap[:, 0], mnist_umap[:, 1], c=y_full, cmap='tab10', s=8)
plt.colorbar()
plt.xlabel('UMAP First Component')
plt.ylabel('UMAP Second Component')
plt.title('UMAP (Full MNIST)')
plt.show()

### Hyperparameter Tuning for UMAP

UMAP's behavior is controlled by several important hyperparameters. We will investigate three primary parameters:

1. **n_neighbors**: Determines the size of the local neighborhood considered. Lower values (5-15) emphasize fine detail and local structure, while higher values (30-100) preserve more global structure. This is conceptually similar to perplexity in t-SNE.

2. **min_dist**: Controls how tightly points are packed in the embedding. Lower values (0.0-0.1) create dense, tightly clustered visualizations, while higher values (0.5-0.99) produce more dispersed layouts with greater separation.

3. **metric**: The distance function used to measure similarity in the original high-dimensional space. Different metrics capture different notions of similarity: Euclidean for overall magnitude, Manhattan for axis-aligned patterns, Cosine for directional similarity, and Correlation for pattern similarity regardless of scale.

In [ ]:
%%time
# UMAP n_neighbors sweep on full dataset (may be slow)
n_neighbors = [5, 10, 20, 50]

plt.figure(figsize=(20, 10))
for i, n_neighbor in enumerate(n_neighbors):
    plt.subplot(2, 2, i+1)
    umap2 = umap.UMAP(n_components=2, random_state=42, n_neighbors=n_neighbor)
    mnist_umap = umap2.fit_transform(X_full)
    plt.scatter(mnist_umap[:, 0], mnist_umap[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('UMAP First Component')
    plt.ylabel('UMAP Second Component')
    plt.title(f'UMAP n_neighbors={n_neighbor}')
plt.show()

In [ ]:
%%time
# UMAP min_dist sweep on full dataset
min_dists = [0.1, 0.25, 0.5, 0.99]

plt.figure(figsize=(20, 10))
for i, min_dist in enumerate(min_dists):
    plt.subplot(2, 2, i+1)
    umap3 = umap.UMAP(n_components=2, random_state=42, min_dist=min_dist)
    mnist_umap = umap3.fit_transform(X_full)
    plt.scatter(mnist_umap[:, 0], mnist_umap[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('UMAP First Component')
    plt.ylabel('UMAP Second Component')
    plt.title(f'UMAP min_dist={min_dist}')
plt.show()

In [ ]:
%%time
# UMAP metric sweep on full dataset
metrics = ['euclidean', 'manhattan', 'cosine', 'correlation']

plt.figure(figsize=(20, 10))
for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    umap4 = umap.UMAP(n_components=2, random_state=42, metric=metric)
    mnist_umap = umap4.fit_transform(X_full)
    plt.scatter(mnist_umap[:, 0], mnist_umap[:, 1], c=y_full, cmap='tab10', s=8)
    plt.colorbar()
    plt.xlabel('UMAP First Component')
    plt.ylabel('UMAP Second Component')
    plt.title(f'UMAP metric={metric}')
plt.show()


**Key takeaways:**

t-SNE excels at revealing local cluster structure but can be computationally expensive. UMAP provides a good balance between computational efficiency and preservation of both local and global structure. The choice of method and hyperparameters should be guided by your specific goals and computational constraints.

## References

This lab is adapted from the following resources:

**t-SNE and UMAP Tutorials:**
* Smendowski (2021). *Data Embedding and Visualization: MDS and t-SNE projections*  
  GitHub: [Data Embedding Repository](https://github.com/Smendowski/data-embedding-and-visualization)

* *Visualizing Neural Networks using t-SNE and UMAP*  
  Kaggle: [Notebook Tutorial](https://www.kaggle.com/code/lzs0047/visualizing-neural-networks-using-t-sne-and-umap/)

**Original Papers:**
* van der Maaten, L., & Hinton, G. (2008). Visualizing Data using t-SNE. *Journal of Machine Learning Research*, 9, 2579-2605.

* McInnes, L., Healy, J., & Melville, J. (2018). UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction. *arXiv preprint arXiv:1802.03426*.

**Additional Resources:**
* [UMAP Documentation](https://umap-learn.readthedocs.io/)
* [scikit-learn Manifold Learning Guide](https://scikit-learn.org/stable/modules/manifold.html)